In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from IPython import embed
import pickle
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.nn.functional as F #functions
from torchvision import datasets, transforms
import sklearn.metrics as sk_m
from sklearn.metrics import plot_confusion_matrix
import os
import shutil

import sys
sys.path.append(os.path.abspath("/Users/andyvarner/Documents/NN_Spring2023/project_2"))
import dataset_methods

%matplotlib notebook

/Users/andyvarner/mambaforge/envs/dev/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/andyvarner/mambaforge/envs/dev/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <E7E99FB4-837B-39DF-9112-617A7DBD769D> /Users/andyvarner/mambaforge/envs/dev/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <A3DB6B6A-7FF5-3593-A8F9-22FFB1D3923A> /Users/andyvarner/mambaforge/envs/dev/lib/python3.8/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [77]:
class ChessRNN(torch.nn.Module): #inherits from nn.Module
    
    #def __init__(self, input_size, hidden_size, num_layers, lr, output_size):
    def __init__(self, input_size, output_size, hidden_dim, num_layers, lr):
        # hidden_dim = the number of features in the hidden state h
        # num_layers = Number of recurrent layers. 
        #              E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked 
        #              RNN, with the second RNN taking in outputs of the first RNN and computing the final 
        #              results. Default: 1
        # nonlinearity = The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
        
        super(ChessRNN, self).__init__() #initialize nn.Module
        
        self.rnn = nn.RNN(input_size, hidden_dim, num_layers=num_layers, batch_first=True,)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lr = lr
        
    def forward(self, x, num_turns):
        
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        out = out[0][num_turns-1]
        
        return out
        
    def init_optimizer(self):
        
        self.optimizer = torch.optim.Adam(self.parameters(), lr = self.lr)

#     def objective(self, outputs, labels): # this is the loss function

#         loss = torch.nn.CrossEntropyLoss(outputs, labels)
        
#         return loss
    
    def init_hidden(self, batch_size):
        
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
    
def train_rnn(model, train_dataset, test_dataset, num_epochs = 10, rate = 5):
    
    criterion = nn.CrossEntropyLoss()
    
    training_losses, valid_losses, train_labels, train_preds, cf_matrices = [], [], [], [], []

    model.init_optimizer()
   
    for epoch in range(num_epochs):
        train_epoch_loss, train_epoch_acc, valid_epoch_loss, valid_epoch_acc = 0, 0, 0, 0

        model.train()        # Set model to train mode

        # Loop over the train data.
        for i, (turns, winner, num_turns), in enumerate(tqdm(train_dataset, desc = "Train Epoch %s" % epoch)): ### turns = inputs, winner = labels

            # Zero out gradients #
            model.optimizer.zero_grad()
            
            # Forward pass #
            outputs = model(turns, num_turns)
            
            #loss = criterion(outputs, winner.float())
            loss = criterion(outputs, torch.argmax(winner, dim=1))  # is this actually correct?
            
            # Backward pass #
            loss.backward()
            model.optimizer.step()
            
            # update training loss #
            train_epoch_loss += loss.item()
            
            
            # Update training accuracy
            train_preds = F.softmax(outputs, dim=1)
            _, train_pred_labels = torch.max(train_preds, dim=1)
            train_epoch_acc += torch.sum(train_pred_labels == torch.argmax(winner, dim=1)).item() 
            
#             train_preds.append(winner.numpy())
#             train_labels.append(winner)


        train_loss = train_epoch_loss / len(train_dataset)
        train_acc = train_epoch_acc / len(train_dataset)
        training_losses.append(train_loss)
        train_labels.append(winner)
        train_preds.append(pred_labels)

        
        ###############
        
        print(f"epoch {epoch}, epoch_loss={train_loss}, epoch_acc={train_acc}")

        # Validate network
        
        if(epoch % rate == 0):
            
            model.eval()
            
            valid_epoch_loss, valid_epoch_acc = 0, 0
            valid_labels, valid_preds = [], []

            for i, (turns, winner, num_turns), in enumerate(tqdm(test_dataset, desc = "Test Epoch %s" % epoch)):
                
                outputs = model(turns, num_turns)
                
                loss = criterion(outputs, torch.argmax(winner, dim=1))
                valid_epoch_loss += loss.item()
                
                valid_preds.append(winner.numpy())
                valid_labels.append(winner)
                
                # Update validation accuracy
                preds = F.softmax(outputs, dim=1)
                _, pred_labels = torch.max(preds, dim=1)
                valid_epoch_acc += torch.sum(pred_labels == torch.argmax(winner, dim=1)).item()
                
                valid_labels.append(torch.argmax(winner, dim=1))
                valid_preds.append(pred_labels)

            # Calculate average validation loss and accuracy
#             print("figure out difference between games_20 and games_100:")
#             print(f"games_100: shape: {test_dataset.shape}, type: {type(test_datset)}")

            valid_losses.append(valid_epoch_loss / (len(test_dataset) + 1e-3))
            valid_acc = valid_epoch_acc / (len(test_dataset) + 1e-3)
            
            # Calculate confusion matrix
            #cf_matrix = sk_m.confusion_matrix(torch.cat(valid_labels).numpy(), torch.cat(valid_preds).numpy())
            #cf_matrices.append(cf_matrix)

            print("Valid Accuracy %s" % valid_acc)
                
#                 #if(pred == label):
#                 if(torch.argmax(winner) == torch.argmax(outputs)):
#                     valid_epoch_acc += 1
                    
#             valid_epoch_acc = valid_epoch_acc / (i + 1)
            
#             print("Valid Accuracy %s" % valid_acc)
                
    
                ##get metrics
            cf_matrix = sk_m.confusion_matrix(valid_labels, valid_preds)
            
            #epoch_accuracy = calculate_accuracy(np.asarray(all_preds), np.asarray(all_labels))

            cf_matrices.append(cf_matrix)
            print(f"confusion matrix appended. epoch {epoch}")
            model.train()

    
    
    
    
        ##get metrics
        training_metrics = {}
        #cf_matrix = sk_m.confusion_matrix(valid_labels, valid_preds)

#         epoch_accuracy = calculate_accuracy(np.asarray(valid_preds), np.asarray(valid_labels))

        #cf_matrices.append(cf_matrix)
        #print(f"confusion matrix appended. epoch {epoch}")
        model.train()
            
        metrics = {}
        metrics["valid_losses"] = valid_losses
        metrics["training_losses"] = training_losses
        metrics["labels"] = valid_labels
        metrics["preds"] = valid_preds
        metrics["mats"] = cf_matrices

    #return training_loss, training_metrics
    return metrics

In [78]:
# Set hyperparameters
input_size = 5    # number of features in the input (shape of `moves` encoding)
output_size = 2   # number of output classes ([white, black], 0-loser, 1-winner)
hidden_dim = 12  # number of hidden units in the RNN
num_layers = 4

learning_rate = 0.001
num_epochs = 10

model = ChessRNN(input_size, output_size, hidden_dim, num_layers, learning_rate)

filename = "games_20.pkl"

train, test = dataset_methods.get_dataset(filename)
# embed()

train_metrics = train_rnn(model, train, test, num_epochs=num_epochs)

Train Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████| 1385/1385 [00:09<00:00, 149.72it/s]


epoch 0, epoch_loss=0.44800033247105053, epoch_acc=0.8252707581227436


Test Epoch 0: 0it [00:00, ?it/s]


Valid Accuracy 0.0
confusion matrix appended. epoch 0


Train Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████| 1385/1385 [00:09<00:00, 150.24it/s]


...

epoch 7, epoch_loss=0.34490783011897164, epoch_acc=0.8577617328519855


Train Epoch 8: 100%|████████████████████████████████████████████████████████████████████████████████████| 1385/1385 [00:09<00:00, 139.43it/s]


epoch 8, epoch_loss=0.3404392678626823, epoch_acc=0.8592057761732852


Train Epoch 9: 100%|████████████████████████████████████████████████████████████████████████████████████| 1385/1385 [00:09<00:00, 143.68it/s]

epoch 9, epoch_loss=0.33771941479066864, epoch_acc=0.8592057761732852


In [79]:
train_metrics.keys()
#train_metrics["valid_losses"]

dict_keys(['valid_losses', 'training_losses', 'labels', 'preds', 'mats'])

In [80]:
train_metrics['preds']

[]

In [ ]:
results = {}
results["loss"] = train_loss
results["validation"] = train_metrics # 1 confusion matrix per validation run
    
path_save = f"/Users/andyvarner/Documents/NN_Spring2023/project_2/Results"

# if(os.path.join(path_save)):
#     print("creating folder")
#     create_folder(path_save)  

title = "%s" % (str(filename).zfill(3))
file_save = os.path.join(path_save, title)

pickle.dump(results, open(file_save, "wb"))

# torch.save(model.state_dict(), "kernel_size_1.pt")

In [ ]:
############### GRAVEYARD ################

def train_rnn(model, train_dataset, test_dataset, num_epochs = 10, rate = 5):
    
    criterion = nn.CrossEntropyLoss()
    
    training_loss, valid_loss, cf_matrices = [], [], []
    train_labels, train_preds = [], []

    model.init_optimizer()
   
    for epoch in range(num_epochs):
        train_epoch_loss = 0
        train_epoch_acc = 0
        
        valid_epoch_loss = 0
        valid_epoch_acc = 0

        # Set model to train mode
        model.train()

        # Loop over the train data
        
        for i, data, in enumerate(tqdm(train_dataset, desc = "Train Epoch %s" % epoch)):
            
            # Zero out gradients
            model.optimizer.zero_grad()

            turns = data[0]  #inputs
            winner = data[1] #labels
            num_turns = data[2]
            
            # Forward pass
            outputs = model(turns, num_turns)

#             print(f"pred = {outputs}")
#             print(f"pred = {torch.argmax(outputs)}")
#             print(f"label = {winner}")
#             print(f"label = {torch.argmax(winner)}")
#             print(" ")
            
            #loss = criterion(outputs, winner.float())
            loss = criterion(outputs, torch.argmax(winner, dim=1))  # is this actually correct?
            
            # Backward pass
            loss.backward()
            
            model.optimizer.step()

#             ###### Update metrics  my code below:
#             if(torch.argmax(winner) == torch.argmax(outputs)):
#                 train_acc += 1
                    
#             train_acc = train_acc / (i + 1)
            
#             print("Train Accuracy %s" % train_acc)

#            train_epoch_loss += loss.item()

#         train_loss = train_epoch_loss / (i + 1)

#         print(f"epoch {epoch}, epoch_loss={train_loss}")
        ###############
        # Update metrics ######### chatGPT code below:
            train_epoch_acc += torch.sum(torch.argmax(outputs, dim=1) == torch.argmax(winner, dim=1)).item()
            
            # update training loss
            train_epoch_loss += loss.item()
            
            # Update training accuracy
            preds = F.softmax(outputs, dim=1)
            _, pred_labels = torch.max(preds, dim=1)
            train_epoch_acc += torch.sum(pred_labels == torch.argmax(winner, dim=1)).item()

        #train_loss = train_epoch_loss / (i + 1)
        train_acc = train_acc / ((i + 1) * train_dataset.batch_size)

        training_loss.append(train_loss)
        
        ###############

        # Calculate average training loss and accuracy
        train_loss = train_epoch_loss / len(train_dataset)
        train_acc = train_epoch_acc / len(train_dataset)
        training_loss.append(train_loss)
        train_labels.append(winner)
        train_preds.append(pred_labels)
        
        print(f"epoch {epoch}, epoch_loss={train_epoch_loss}, epoch_acc={train_acc}")

        # Validate network
        
        if(epoch % rate == 0):
            
            model.eval()
            
            valid_acc = 0
            valid_labels, valid_preds = [], []

            for i, data, in enumerate(tqdm(test_dataset, desc = "Test Epoch %s" % epoch)):
                
                turns = data[0]  #inputs
                winner = data[1] #labels
                num_turns = data[2]
                
                outputs = model(turns, num_turns)

                valid_preds.append(winner.numpy())
                valid_labels.append(winner)
                
                #if(pred == label):
                if(torch.argmax(winner) == torch.argmax(outputs)):
                    valid_acc += 1
                    
            valid_acc = valid_acc / (i + 1)
            
            print("Valid Accuracy %s" % valid_acc)
                
        ##get metrics
        training_metrics = {}
        cf_matrix = sk_m.confusion_matrix(valid_labels, valid_preds)

        epoch_accuracy = calculate_accuracy(np.asarray(valid_preds), np.asarray(valid_labels))

        cf_matrices.append(cf_matrix)
        print(f"confusion matrix appended. epoch {epoch}")
        model.train()
            
        training_metrics = {}
        training_metrics["labels"] = valid_labels
        training_metrics["preds"] = valid_preds
        training_metrics["mats"] = cf_matrices

    return training_loss, training_metrics

In [ ]:
# i think the train loop is working:

def train_rnn(model, train_dataset, test_dataset, num_epochs = 10, rate = 5):
    
    criterion = nn.CrossEntropyLoss()
    
    training_losses, valid_losses, train_labels, train_preds, cf_matrices = [], [], [], [], []

    model.init_optimizer()
   
    for epoch in range(num_epochs):
        train_epoch_loss, train_epoch_acc, valid_epoch_loss, valid_epoch_acc = 0, 0, 0, 0

        model.train()        # Set model to train mode

        # Loop over the train data.
        for i, (turns, winner, num_turns), in enumerate(tqdm(train_dataset, desc = "Train Epoch %s" % epoch)): ### turns = inputs, winner = labels

            # Zero out gradients #
            model.optimizer.zero_grad()
            
            # Forward pass #
            outputs = model(turns, num_turns)
            
            #loss = criterion(outputs, winner.float())
            loss = criterion(outputs, torch.argmax(winner, dim=1))  # is this actually correct?
            
            # Backward pass #
            loss.backward()
            model.optimizer.step()
            
            # update training loss #
            train_epoch_loss += loss.item()
            
            
            # Update training accuracy
            preds = F.softmax(outputs, dim=1)
            _, pred_labels = torch.max(preds, dim=1)
            train_epoch_acc += torch.sum(pred_labels == torch.argmax(winner, dim=1)).item() 
            
#             train_preds.append(winner.numpy())
#             train_labels.append(winner)


        train_loss = train_epoch_loss / len(train_dataset)
        train_acc = train_epoch_acc / len(train_dataset)
        training_losses.append(train_loss)
        train_labels.append(winner)
        train_preds.append(pred_labels)

        
        ###############

        # Calculate average training loss and accuracy
        average_training_loss = train_epoch_loss / len(train_dataset)
        average_training_acc = train_epoch_acc / len(train_dataset)
        
        training_losses.append(average_training_loss)
        #train_labels.append(winner)
        #train_preds.append(pred_labels)
        
        print(f"epoch {epoch}, epoch_loss={train_loss}, epoch_acc={train_acc}")